In [24]:
import pandas as pd

bios = pd.read_excel("./data/olympics-data.xlsx")

#results = pd.read_parquet("./data/results.parquet")

#print(results.columns)

display(bios[(bios["height_cm"] > 215) & (bios["born_country"]=="USA")][["name", "height_cm"]])

bios_filtered1 = bios.query('name.str.contains("Patrick|Keith", case=False, na=False)', engine='python')

bios_filtered2 = bios.query('born_country in ["USA", "GBR"] and name.str.contains("Patrick|Keith", case=False, na=False)', engine='python')

bios_filtered3 = bios.query('born_country in ["USA"]')


display(bios_filtered1)
display(bios_filtered2)
display(bios_filtered3)

,name,height_cm
5781,Tommy Burleson,223.0
6722,Shaquille O'Neal,216.0
6937,David Robinson,216.0
123850,Tyson Chandler,216.0


,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date
6,7,Patrick Chila,1969-11-27,Ris-Orangis,Essonne,FRA,France,180.0,73.0,NaN
119,120,Patrick Wheatley,1899-01-20,Vryheid,KwaZulu-Natal,RSA,Great Britain,NaN,NaN,1967-11-05
319,320,Patrick De Koning,1961-04-23,Dendermonde,Oost-Vlaanderen,BEL,Belgium,178.0,92.0,NaN
1897,1907,Keith Hanlon,1966-09-01,NaN,NaN,NaN,Ireland,NaN,NaN,NaN
2115,2125,Patrick Jopp,1962-01-08,NaN,NaN,NaN,Switzerland,176.0,67.0,NaN
...,...,...,...,...,...,...,...,...,...,...
143975,147633,Patrick Chinyemba,2001-01-03,NaN,NaN,NaN,Zambia,NaN,NaN,NaN
144172,147850,Patrick Jakob,1996-10-17,Sankt Johann in Tirol,Tirol,AUT,Austria,NaN,NaN,NaN
144547,148239,Patrick Galbraith,1986-03-11,Haderslev,Syddanmark,DEN,Denmark,NaN,NaN,NaN
144565,148257,Patrick Russell,1993-01-04,Gentofte,Hovedstaden,DEN,Denmark,186.0,93.0,NaN


,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date
3505,3517,Keith Wallace,1961-03-29,Preston,England,GBR,Great Britain,165.0,51.0,1999-12-31
12053,12118,Keith Hervey,1898-11-03,Fulham,England,GBR,Great Britain,NaN,NaN,1973-02-22
14577,14674,Keith Harrison,1933-03-28,Birmingham,England,GBR,Great Britain,NaN,NaN,NaN
16166,16281,Keith Reynolds,1963-12-25,Solihull,England,GBR,Great Britain,173.0,68.0,NaN
18122,18243,Jim Fitzpatrick,1892-06-02,Los Angeles,California,USA,United States,NaN,NaN,1973-10-09
18130,18251,Jack Patrick,1898-11-25,Palo Alto,California,USA,United States,187.0,95.0,1959-05-31
18585,18712,Patrick Austen,1933-03-18,Kingston-upon-Thames,England,GBR,Great Britain,NaN,NaN,2013-10-14
18734,18862,Keith Sinclair,1945-06-26,Sunderland,England,GBR,Great Britain,190.0,79.0,NaN
29897,30123,Keith Langley,1961-06-03,Aldershot,England,GBR,Great Britain,173.0,70.0,NaN
32759,33005,Patrick O'Neill,1956-03-20,Stamford,Connecticut,USA,United States,198.0,95.0,NaN


,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date
54,55,Monique Javer,1967-07-22,Burlingame,California,USA,Great Britain,177.0,64.0,NaN
960,964,Xóchitl Escobedo,1968-09-17,West Covina,California,USA,Mexico,170.0,60.0,NaN
961,965,Angélica Gavaldón,1973-10-03,El Centro,California,USA,Mexico,160.0,54.0,NaN
1231,1238,Bert Schneider,1897-07-01,Cleveland,Ohio,USA,Canada,NaN,NaN,1986-02-20
1345,1352,Laura Berg,1975-01-06,Santa Fe Springs,California,USA,United States,168.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...
145445,149168,Kristen Santos,1994-11-02,Fairfield,Connecticut,USA,United States,NaN,NaN,NaN
145446,149169,Corinne Stoddard,2001-08-15,Seattle,Washington,USA,United States,NaN,NaN,NaN
145454,149180,Anna Hoffmann,2000-03-28,Madison,Wisconsin,USA,United States,NaN,NaN,NaN
145457,149183,Alix Wilkinson,2000-08-02,Mammoth Lakes,California,USA,United States,NaN,NaN,NaN
